In [38]:
import numpy as np
import pandas as pd
pd.pandas.set_option('display.max_columns', None)
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import joblib

In [2]:
features = pd.read_csv('data/training_set_features.csv', index_col="respondent_id")

In [3]:
label = pd.read_csv('data/training_set_labels.csv', index_col="respondent_id")

In [4]:
df = features.merge(label, on='respondent_id', how='inner')

df.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0


In [5]:
def df_summary(df):
    
    print(f"Dataset Shape: {df.shape}")
    
    summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
    
    summary = summary.reset_index()
    
    summary['Name'] = summary['index']
    
    summary = summary[['Name','dtypes']]
    
    summary['Uniques'] = df.nunique().values
    
    summary['Missing Count'] = df.isnull().sum().values
    
    summary['Missing Percentage'] = df.isnull().sum().values/len(df)
    
    summary['Missing Percentage'] = summary['Missing Percentage'].map("{:.2%}".format)
    
    for name in summary['Name'].value_counts().index:
        
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

In [6]:
df_summary(df)

Dataset Shape: (26707, 37)


,Name,dtypes,Uniques,Missing Count,Missing Percentage,Entropy
0,h1n1_concern,float64,4,92,0.34%,1.86
1,h1n1_knowledge,float64,3,116,0.43%,1.33
2,behavioral_antiviral_meds,float64,2,71,0.27%,0.28
3,behavioral_avoidance,float64,2,208,0.78%,0.85
4,behavioral_face_mask,float64,2,19,0.07%,0.36
5,behavioral_wash_hands,float64,2,42,0.16%,0.67
6,behavioral_large_gatherings,float64,2,87,0.33%,0.94
7,behavioral_outside_home,float64,2,82,0.31%,0.92
8,behavioral_touch_face,float64,2,128,0.48%,0.91
9,doctor_recc_h1n1,float64,2,2160,8.09%,0.76


In [7]:
X_num_cols = [var for var in df.columns if df[var].dtype != 'O' and var not in ['h1n1_vaccine', 'seasonal_vaccine']]

In [8]:
X_cat_cols = [var for var in df.columns if df[var].dtype == 'O' and var not in ['h1n1_vaccine', 'seasonal_vaccine']]

In [9]:
X_cols = [var for var in df.columns if var not in ['h1n1_vaccine', 'seasonal_vaccine']]
X = df[X_cols]

In [10]:
y_cols = ['h1n1_vaccine', 'seasonal_vaccine']
y = df[y_cols]

In [11]:
# Categorical features

In [12]:
imp_cat = SimpleImputer(strategy='constant', fill_value='missing')
ohe = OneHotEncoder(handle_unknown='ignore')

In [13]:
pipe_cat = make_pipeline(imp_cat, ohe)

In [14]:
# Numeric features

In [15]:
imp_num = SimpleImputer(strategy='median')

In [16]:
ct = make_column_transformer(
    (pipe_cat, X_cat_cols),
    (imp_num, X_num_cols),
    remainder='passthrough')

In [17]:
# Testing multiple models for a baseline score

In [18]:
clfs = []
clfs.append(LogisticRegression(solver='liblinear', random_state=123))
clfs.append(KNeighborsClassifier(n_neighbors=3))
clfs.append(DecisionTreeClassifier(random_state=123))
clfs.append(RandomForestClassifier(random_state=123))
clfs.append(GradientBoostingClassifier(random_state=123))

for classifier in clfs:
    
    pipe = make_pipeline(ct, MultiOutputClassifier(classifier))
    scores = cross_validate(pipe, X, y, cv=5, scoring='roc_auc')
    print('============================================================')
    print(str(classifier))
    print('')
    print('Results:')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            print(key,' std ', values.std())
    print('============================================================')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Results:
fit_time  mean  0.5831313133239746
fit_time  std  0.049685918327805764
score_time  mean  0.020204639434814452
score_time  std  0.0011666129227261223
test_score  mean  0.8436528120577952
test_score  std  0.004037829485590545
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

Results:
fit_time  mean  1.4719316959381104
fit_time  std  0.03140478760184712
score_time  mean  27.511002492904662
score_time  std  1.2238915424485675
test_score  mean  0.7222192355822674
test_score  std  0.003533913267700397
DecisionTreeClassifier

In [19]:
# estimator = LogisticRegression(solver='liblinear', penalty="l1", C=1, max_iter=1000, random_state=123)

In [20]:
estimator = RandomForestClassifier(n_estimators=200, min_samples_split=4, min_samples_leaf=5,max_features='sqrt',
                                   max_depth=20, bootstrap=False, random_state=123)

In [21]:
estimators = MultiOutputClassifier(estimator)

In [22]:
pipe = make_pipeline(ct, estimators)

In [23]:
# Grid Search

In [24]:
params = {}

# Pre-processing params
params['columntransformer__pipeline__simpleimputer__add_indicator'] = [False, True]
params['columntransformer__simpleimputer__add_indicator'] = [False, True]

# Logreg params
# params['multioutputclassifier__estimator__penalty'] = ['l1', 'l2']
# params['multioutputclassifier__estimator__C'] = [0.1, 1, 10]

# RF params
params["multioutputclassifier__estimator__bootstrap"] = [True, False]
params["multioutputclassifier__estimator__max_features"] = [None, 'sqrt', 'log2']
params["multioutputclassifier__estimator__max_depth"] = [2, 5, 10, 20, 30]
params["multioutputclassifier__estimator__min_samples_leaf"] = [1, 5, 10]
params["multioutputclassifier__estimator__min_samples_split"] = [2, 4, 8, 10, 12]
params["multioutputclassifier__estimator__n_estimators"] = [100, 200, 500, 1000]

In [25]:
grid = RandomizedSearchCV(pipe, params, cv=5, scoring='roc_auc', random_state=123)
grid.fit(X, y);

In [26]:
results = pd.DataFrame(grid.cv_results_)
results.sort_values('rank_test_score').head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_multioutputclassifier__estimator__n_estimators,param_multioutputclassifier__estimator__min_samples_split,param_multioutputclassifier__estimator__min_samples_leaf,param_multioutputclassifier__estimator__max_features,param_multioutputclassifier__estimator__max_depth,param_multioutputclassifier__estimator__bootstrap,param_columntransformer__simpleimputer__add_indicator,param_columntransformer__pipeline__simpleimputer__add_indicator,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
23,11.409973,0.138094,0.314871,0.004957,200,4,5,sqrt,20,False,True,False,{'multioutputclassifier__estimator__n_estimato...,0.860426,0.855612,0.861526,0.867415,0.857032,0.860402,0.004117,1
7,16.311478,0.112614,0.822785,0.001625,500,8,1,log2,30,True,True,True,{'multioutputclassifier__estimator__n_estimato...,0.856066,0.852904,0.858885,0.864534,0.854336,0.857345,0.004110,2
0,18.790437,0.236381,0.662949,0.001470,500,2,10,log2,30,False,True,False,{'multioutputclassifier__estimator__n_estimato...,0.856508,0.851590,0.858725,0.862885,0.853151,0.856572,0.004025,3
20,13.902735,0.080999,0.388088,0.001674,200,2,1,sqrt,30,False,True,False,{'multioutputclassifier__estimator__n_estimato...,0.855729,0.852830,0.857616,0.863504,0.852683,0.856473,0.003973,4
12,6.322423,0.027330,0.335095,0.001021,200,12,1,log2,30,True,True,True,{'multioutputclassifier__estimator__n_estimato...,0.854801,0.852027,0.857098,0.863240,0.853962,0.856226,0.003867,5
10,6.036506,0.100829,0.226904,0.001601,200,12,5,sqrt,10,True,True,True,{'multioutputclassifier__estimator__n_estimato...,0.854757,0.850632,0.857011,0.861857,0.851292,0.855110,0.004096,6
5,14.620896,0.486105,0.520118,0.007977,500,2,10,sqrt,10,True,True,True,{'multioutputclassifier__estimator__n_estimato...,0.855212,0.850015,0.857283,0.861563,0.851253,0.855065,0.004176,7
6,7.909983,0.151092,0.288265,0.000980,200,4,10,log2,30,False,True,True,{'multioutputclassifier__estimator__n_estimato...,0.854375,0.849906,0.856728,0.861214,0.851194,0.854684,0.004048,8
25,4.403793,0.125254,0.130230,0.001166,100,10,5,sqrt,10,False,True,True,{'multioutputclassifier__estimator__n_estimato...,0.854651,0.849562,0.856449,0.861432,0.850435,0.854506,0.004307,9
13,14.741186,0.172603,0.513241,0.002739,500,2,1,log2,10,False,True,True,{'multioutputclassifier__estimator__n_estimato...,0.850068,0.845436,0.853116,0.856435,0.846685,0.850348,0.004058,10


In [27]:
print(F"Grid best score: {grid.best_score_}")
print('')
print(F"Grid best score: {grid.best_params_}")

Grid best score: 0.8604020094544893

Grid best score: {'multioutputclassifier__estimator__n_estimators': 200, 'multioutputclassifier__estimator__min_samples_split': 4, 'multioutputclassifier__estimator__min_samples_leaf': 5, 'multioutputclassifier__estimator__max_features': 'sqrt', 'multioutputclassifier__estimator__max_depth': 20, 'multioutputclassifier__estimator__bootstrap': False, 'columntransformer__simpleimputer__add_indicator': True, 'columntransformer__pipeline__simpleimputer__add_indicator': False}


In [41]:
joblib.dump(grid.best_estimator_, 'grid_best_random_forest.joblib')

['grid_best_random_forest.joblib']

In [28]:
test_features_df = pd.read_csv('data/test_set_features.csv', index_col="respondent_id")

In [43]:
test_probas = grid.predict_proba(test_features_df)

In [44]:
submission_df = pd.read_csv('data/submission_format.csv', index_col="respondent_id")

In [45]:
submission_df.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.5,0.7
26708,0.5,0.7
26709,0.5,0.7
26710,0.5,0.7
26711,0.5,0.7


In [46]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_features_df.index.values, submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = test_probas[0][:, 1]
submission_df["seasonal_vaccine"] = test_probas[1][:, 1]

submission_df.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.168637,0.333359
26708,0.062044,0.062787
26709,0.280432,0.765519
26710,0.571955,0.839227
26711,0.252667,0.492775


In [33]:
submission_df.to_csv('results/20200501_submission_rf.csv', index=True)